# GAI Final Project - Judgment Summarization

## 第28組
## 組長: 趙昌昱
## 組員: 吳瀚惟 陳諺樟 鄧新泰




## 安裝套件


In [1]:
! pip install transformers
# ! pip install --upgrade paddlenlp>=2.0.0rc -i https://pypi.org/simple
! pip install datasets
! pip install torcheval
! pip install scikit-learn
! pip install torch
! pip install pytorch-ignite
! pip install ipywidgets
! pip install jieba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_6

In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from ignite.metrics import Rouge
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import jieba
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
! pip freeze > requirements.txt

## 超參數
 - 學習率 (learning rate): 1e-4
 - 訓練輪數 (epochs): 10
 - 優化器 (optimizer): AdamW
 - 批次大小 (batch size): 8
 - 評量指標 (evaluation matrics)Rouge-2

In [ ]:
lr = 1e-4
epochs = 10
train_batch_size = 1
validation_batch_size = 1
test_batch_size = 1
random_seed = 42
max_length = 300

# path setting that souldn't be changed
tokenizer_path = "./saved_tokenizer"
dataset_path = "./train.xlsx"
test_dataset_path = './test.xlsx'
model_path = "./saved_model"

## 載入BART模型
 - 使用huggingface裝載模型的架構、參數和tokenizer
 - 保存在路徑./cache/中
 - 用.to(device)把模型裝載入訓練設備(GPU)
 - 使用的 tokenizer 請儲存到指定路徑

In [ ]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

model = MT5ForConditionalGeneration.from_pretrained("heack/HeackMT5-ZhSum100k").to(device)
tokenizer = T5Tokenizer.from_pretrained("heack/HeackMT5-ZhSum100k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/751 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
optimizer = AdamW(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

## 資料處理
 - 使用 torch.utils.data 中的 Dataset 和 Dataloader 成批次地讀取和預處理資料

In [ ]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = tokenizer([each["origin_context"] for each in sample], max_length=1892 ,padding=True, truncation=True, return_tensors="pt")
    model_outputs = tokenizer([each["summary"] for each in sample], max_length=1892 ,padding=True, truncation=True, return_tensors="pt")
    return model_inputs["input_ids"].to(device), model_outputs["input_ids"].to(device)

class Legal_Judgment_Dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        data_df = pd.read_excel(dataset_path)
        self.data = []
        for index, row in data_df.iterrows():
            origin_context = "summary: " + str(row['裁判原文'])
            summary = row['摘要']
            self.data.append({"origin_context": origin_context, "summary": summary})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)


- 在範例程式中有切分成 train, valid, test，實作上可將全部資料用於 train 與 valid

In [ ]:
dataset = Legal_Judgment_Dataset()
train_dataset, validation_dataset = train_test_split(dataset, test_size=0.2, random_state=random_seed)
summary_train = DataLoader(train_dataset, collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
summary_validation = DataLoader(validation_dataset, collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)

rouge = Rouge(variants=["L", 2], multiref="best")
print(f"Dataset example: \n{train_dataset[0]} \n{train_dataset[1]} \n{train_dataset[2]}")

Dataset example: 
{'origin_context': 'summary: 最高法院刑事判決一○五年度台上字第三三九七號上訴人張嘉如（原名張書平）選任辯護人黃俊嘉律師上訴人許洺瑋王祥益馮鑰仁（原名馮文軍）共同選任辯護人劉嵐律師上訴人林碧玟林巾愉（原名林碧容）共同選任辯護人鍾義律師上訴人陳沛彤邱歆茹張勝鈞（原名張依濱）林君徽陳紋慧黃景微宋珮青（原名宋淑如）凌偉婷陳昕婕（原名陳雅惠）林家瑀（原名林麗娟）張靖梅上列上訴人等因違反證券交易法案件，不服台灣高等法院高雄分院中華民國一○四年十一月二十三日第二審更審判決（一○三年度重金上更㈠字第五號，起訴案號：台灣高雄地方法院檢察署九十七年度偵字第七一○七號，九十八年度偵字第一二六三九、一五五二四、二二二二三號），提起上訴，本院判決如下：主文原判決關於張勝鈞、張嘉如、許洺瑋、王祥益、馮鑰仁、林碧玟、林巾愉、陳沛彤、邱歆茹、林君徽、陳紋慧、黃景微、宋珮青、凌偉婷、陳昕婕、林家瑀、張靖梅部分均撤銷，發回台灣高等法院高雄分院。理由一、本件原判決認定張勝鈞（原名張依濱）、張嘉如（原名張書平）、許洺瑋、王祥益、馮鑰仁（原名馮文軍）、林碧玟、林巾愉（原名林碧容）、陳沛彤、邱歆茹、林君徽、陳紋慧、黃景微、宋珮青（原名宋淑如）、凌偉婷、陳昕婕（原名陳雅惠）、林家瑀（原名林麗娟）、張靖梅（以上十七人，下稱張勝鈞等十七人）有其事實欄所載之犯行，因而撤銷第一審關於張勝鈞等十七人犯證券交易法第一百七十一條第一項第一款之證券詐偽罪（下稱證券詐偽罪）部分之判決，改判均從一重論處張勝鈞等十七人共同犯證券詐偽罪刑（皆想像競合犯證券交易法第一百七十五條第一項之非法經營證券業務罪），固非無見。二、惟查：㈠、刑事訴訟法第一百五十九條之五規定之傳聞例外，乃基於當事人進行主義中之處分主義，藉由當事人等「同意」之此一處分訴訟行為，與法院之介入審查其適當性要件，將原不得為證據之傳聞證據，賦予其證據能力。本乎程序之明確性，其第一項「經當事人於審判程序同意作為證據」者，當係指當事人意思表示無瑕疵可指之明示同意而言，以別於第二項之當事人等「知而不為異議」之默示擬制同意。當事人已明示同意作為證據之傳聞證據，並經法院審查其具備適當性之要件者，若已就該證據實施調查程序，即無許當事人再行撤回同意之理，以維訴訟程序安定性、確實性之要求。所謂「經當事人於審判程序同意作為證據」

## 驗證
驗證程式
 - 將驗證資料輸入模型，用Rouge-2評價輸出的效果
 - Rouge的使用方法參考 https://pytorch.org/ignite/generated/ignite.metrics.Rouge.html

In [ ]:
def evaluate(model, dataset):
    pbar = tqdm(dataset)
    pbar.set_description(f"Evaluating")
    loss_list = []
    for inputs, targets in pbar:
        loss = model(inputs, labels=targets).loss
        loss_list.append(loss.item())
        pbar.set_postfix(loss = loss.item())
        outputs = [each.replace("<unk>", "").replace("<pad>","") for each in tokenizer.batch_decode(model.generate(inputs, max_new_tokens=2048, no_repeat_ngram_size=2), skip_special_tokens=True)]
        targets = [each.replace("<unk>", "").replace("<pad>","") for each in tokenizer.batch_decode(targets, skip_special_tokens=True)]
        print("model generate examples:")
        print(f"output: {outputs}")
        print(f"target: {targets}")
        for out, tar in zip(outputs, targets):
            sentence = " ".join(jieba.cut(out)).split()
            ground_truth = " ".join(jieba.cut(tar)).split()
            for s in sentence:
                rouge.update(([s], [ground_truth]))

    return rouge.compute(), np.mean(np.array(loss_list))



## 訓練
 - 將資料成批次輸入BART模型，並獲取其損失函數數值，隨後計算梯度優化
 - tqdm用來顯示模型的訓練進度

In [ ]:
for ep in range(epochs):
    model.train()
    pbar = tqdm(summary_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        loss = model(input_ids=inputs, labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    torch.save(model, f'{model_path}/ep{ep}.mod')
    model.eval()
    validation_score, validation_loss = evaluate(model, summary_validation)
    scheduler.step(validation_loss)
    print(f"Rouge-2 score on epoch {ep}:", validation_score)

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

model generate examples:
output: ['一、按政府債權法第5條及第9條第1項規定,政府機關不得提供政府財物,其所為之研究均非以人為對象,而非人體採驗等語,即屬無稽云云。是被上訴人以病患死後即成社會上所稱之「人」,故政府債務人於行政救濟程序上核無不合。']
target: ['政府資訊,固不分政府機關係基於公權力行政或私經濟行政而作成或取得,然須係政府機關職權範圍內已作成或取得,而以政府資訊公開法第3條規定之方式存在者,始屬該法規範之對象,是雖屬政府機關職權範圍內業務,然機關未有以政府資訊公開法第3條所規定之形式存在之資訊時,該法並未賦與人民得請求政府機關作成之權利,政府機關亦無應其要求作成政府資訊之義務。又人民申請提供政府資訊時,受申請機關除有政府資訊公開法第18條所規定,應限制公開或不予提供之情形外,即應提供。又政府機關對人民申請提供、更正或補充政府資訊所為之決定,具行政處分性質,申請人不服得提起課予義務訴訟請求救濟。']
model generate examples:
output: ['按行政訴訟法第242條定有明文。依同法第244條第1項規定,判決不適用法規或適用不當者,為違背法令;而判決有同條(第2項)所列各款情形為理由時,其上訴狀或理由書應有具體之敘明,並揭示該法條之條項或其內容;若係成文法以外之法則,應發掘司法院解釋或本院之判例,則應揭認該判解之字號或之內容。如以行政判決法第233條規定不合時 即難認為已對智慧財產法院行政處之證據不備理由之違法。且若原判決認定系爭商標與據以異議諸商標之外觀則屬「圖樣差異甚大,案情互殊,自無法比附援引」?其亦顯有判決理由前後矛盾,及違誤經驗法論及論理法認之處,當屬判決違後法規定云云。']
target: ['一、依智慧財產案件審理法第32條規定提起上訴者,除有特別規定外,依同法第1條規定,應適用行政訴訟法關於上訴審程序相關規定。又對於高等行政法院判決之上訴,非以其違背法令為理由,不得為之,行政訴訟法第242條定有明文。依同法第243條第1項規定,判決不適用法規或適用不當者,為違背法令;而判決有同條第2項所列各款情形之一者,為當然違背法令。是當事人對於智慧財產法院行政判決上訴,如依行政訴訟法第243條第1項規定,以判決有不適用法規或適用不當為理由時,其上訴狀或理由書應有具體之指摘,並揭示該法

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

...

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

model generate examples:
output: ['按政府情況採取政府機關提供政府數者,依政府詢願法第7條及第9條規定,政府財產為限,其所為之研究均非以人為對象。而上訴人於104年1月21日之申請,應作成准予預留系爭人員等任職於被上開人期間之工作物,「進行無心跳器官捐贈摘取之次數及各次時」,以及「此些『以人在處象」經研究終局後施行之日期,即非無需申請或經由人體試驗許可。......四、政府訴訟法第3條、第5條第1項前段,第10條所列舉之10項行為,並非行政處分。 二、行政法院依其立法理由及法律理由,以:政府債產生效力之「生命或身體」之目的,在提升我國健康及健康之生命力之上訴者之功能,不因其無法提供或接受之,而非有必要之限制。是原審就此不服,循序提起本件訴訟,程序上核無不合。']
target: ['政府資訊,固不分政府機關係基於公權力行政或私經濟行政而作成或取得,然須係政府機關職權範圍內已作成或取得,而以政府資訊公開法第3條規定之方式存在者,始屬該法規範之對象,是雖屬政府機關職權範圍內業務,然機關未有以政府資訊公開法第3條所規定之形式存在之資訊時,該法並未賦與人民得請求政府機關作成之權利,政府機關亦無應其要求作成政府資訊之義務。又人民申請提供政府資訊時,受申請機關除有政府資訊公開法第18條所規定,應限制公開或不予提供之情形外,即應提供。又政府機關對人民申請提供、更正或補充政府資訊所為之決定,具行政處分性質,申請人不服得提起課予義務訴訟請求救濟。']
model generate examples:
output: ['按行政訴訟法第242條定有明文。依同法第244條第1項規定,判決不適用法規或適用不當者,為違背法令;而判決有同條法第2項所列各款情形為理由時,其上訴狀或理由書,應揭示合於該條項全部款之事實。上開規定及說明,即難認為已對智慧財產法院行政判決之違誤法命令有具體之敘明。']
target: ['一、依智慧財產案件審理法第32條規定提起上訴者,除有特別規定外,依同法第1條規定,應適用行政訴訟法關於上訴審程序相關規定。又對於高等行政法院判決之上訴,非以其違背法令為理由,不得為之,行政訴訟法第242條定有明文。依同法第243條第1項規定,判決不適用法規或適用不當者,為違背法令;而判決有同條第2項所列各款情形之一者,為當然違背法令。是當事人對於智慧

  0%|          | 0/188 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/saved_model/mt5.mod /content/drive/MyDrive/mt5.mod

## 測試
- 使用以下程式接跑 test data

In [ ]:
def test(model, test_dataset):
    model.eval()
    pbar = tqdm(test_dataset)
    pbar.set_description(f"Testing")

    for idx, testdata in tqdm(test_dataset.iterrows(), total=test_dataset.shape[0]):
        ori_data = "summary: " + str(testdata["裁判原文"])
        inputs = tokenizer(ori_data, padding=True, truncation=True, return_tensors="pt").to(device)
        outputs_context = tokenizer.decode(model.generate(**inputs)[0], max_length=max_length)
        outputs = outputs_context.replace("<unk>", "").replace("<pad>","")
        test_dataset.loc[idx, "摘要"] = outputs

    return test_dataset


In [ ]:
test_dataset = pd.read_excel(test_dataset_path)
test_dataset['摘要'] = None
result_data = test(model, test_dataset)
result_data.to_excel("result.xlsx", index=False)